%matplotlib inline

번역 출처 : https://dgkim5360.tistory.com/entry/deep-learning-for-nlp-with-pytorch-tutorial-part-2

Word Embeddings: Encoding Lexical Semantics
===========================================

https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#sphx-glr-beginner-nlp-word-embeddings-tutorial-py

Word embedding은 실수(real numbers)로 꽉 들어찬(dense) vector로, 단어장의 단어 하나마다 한 원소를 갖고 있다. NLP 문제를 다루는 대부분의 경우 우리의 feature는 단어일 것이다. 하지만 컴퓨터로 하여금 어떻게 단어를 표시하게 할 것인가? 단어의 ASCII 표현을 사용할 수도 있겠으나, 그것은 단어가 무엇인지 알려줄 뿐 어떤 의미를 갖고 있는지는 말해주지 않는다(아마도 끝글자들을 보고 품사를 안다거나, 대소문자 구분을 통해 뭔가 단어의 성격을 알 수도 있겠으나, 그 조차도 확실하지 않다). 더욱이, 단어들의 표현을 어떤 방식으로 조합할 수 있는 지는 더 어려운 문제이다. $V$가 단어장이고 input이 $|V|$차원이라면, 우리는 종종 neural network을 통해서 꽉 찬 output을 받고 싶은데 output의 차원이 몇 되지 않는 경우가 있다(예를 들어 예측하고자 하는 것이 두 세가지 라벨이라면). 어떻게 하면 엄청나게 큰 차원에서 더 작은 차원으로 dense output을 얻을 수 있을까?

ASCII 표현 대신 one-hot encoding을 사용하면 어떨까? One-hot encoding은 단어 $w$를 이렇게 표시하는 것이다.

\begin{align}\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}\end{align}

여기서 1은 단어 w의 위치에 있다. 단어장에 있는 다른 단어들도 어딘가 1인 자리가 있고 나머지는 0인 vector가 된다.

이 표현 방법은 그 크기가 거대하다는 점 외에 또 큰 단점이 있는데, 그것은 이 표현이 모든 단어들을 서로 간의 관계를 전혀 고려하지 않고 하나 하나 단어만 독립적으로 표현한다는 점이다. 우리가 여기서 정말로 원하는 것은 단어 간의 유사성(similarity)이다. 왜? 예제를 통해 설명하겠다.

우리는 language model을 구축한다고 가정하고, 아래와 같은 문장을 training 데이터로 받았다.

* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

이제 training 데이터에서 못 봤던 새로운 문장 하나가 들어왔다.

* The physicist solved the open problem.

우리의 language model이 이 문장에 대해서 괜찮은 결과를 낼 수도 있지만, 다음 두 가지 사실을 이용한다면 훨씬 그 결과가 좋아질 수 있지 않을까.

* We have seen  mathematician and physicist in the same role in a sentence. Somehow they
  have a semantic relation.
* We have seen mathematician in the same role  in this new unseen sentence
  as we are now seeing physicist.


그러면 새로운 문장에서 physicist가 정말로 적합한 단어라고 생각할 수 있을 것 같다. 이것이 여기서 이야기 하고자 하는 유사성이다. 의미적인 유사성(semantic similarity)이지, 비슷한 철자 표현이 아니다. 이것은 우리가 관측한 것과 모르는 것 사이를 연결해서 언어 데이터의 sparsity를 극복하기 위한 기술이다. 물론 위 예제는 "비슷한 문맥에서 등장하는 단어들은 서로 의미적으로 연관되어 있다"는 언어학적인 기본 전제를 필요로 한다. 이를 [distributional hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics)라고 말한다.

Getting Dense Word Embeddings
===========================================

우리는 이 문제를 어떻게 해결할 수 있을까? 즉, 단어 간의 의미적 유사성을 어떻게 측정하고 코드화할 수 있을까? 우리는 의미에 맞는 성질(semantic attribute)들을 만들어 볼 수도 있겠다. 이미 training 데이터에서 mathematician과 physicist 둘 다 달릴 수 있으므로, "is able to run"이라는 성질에 높은 점수를 주는 것이다.

만약 attribute 하나를 차원 하나에 할당한다면, 우리는 단어 하나에 attribute vector 하나를 아래처럼 연결할 수 있겠다.

\begin{align}q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]\end{align}

\begin{align}q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]\end{align}

이러고 나면 유사성의 정도를 이렇게 계산할 수 있겠다.

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}\end{align}

하지만 여기서 크기를 1로 normalize하는 것이 흔한 관례이다.

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
   {\| q_\text{physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)\end{align}

여기서 $\phi$는 두 벡터 사이의 각도이다. 이 방법대로라면, 엄청나게 비슷한 단어들은 similarity 1을 가질테고, 엄청나게 다른 단어들은 similarity -1를 가지게 될 것이다.

처음에 소개했던 sparse한 one-hot 벡터를 방금 소개한 attribute vector의 특별한 경우로 생각할 수 있다. 이 경우 각 단어는 다른 단어들에 대해 모두 similarity 0를 갖게 될 것이고, 그 각 단어에 대해 유일한 semantic attribute를 부여한 셈이다. 아무튼 새로운 semantic attribute vector는 dense하다 (0인 원소들이 거의 없다).

하지만 반대로 이렇게 새로 만들어지는 vector는 엄청난 고통 그 자체다. Similarity를 측정하기 위해 우리가 굉장히 노력해서 수 천개의 semantic attribute을 생각해냈다 한들, 그 attribute 값을 얼마나로 정할 수 있을까? 프로그래머가 직접 그것들을 설계하는 대신 neural network가 스스로 feature의 표현을 배운다는 것은 딥러닝 아이디어의 핵심이다. 그러니 word embedding이 그저 우리 model의 parameter가 돼서 학습 과정에서 업데이트되면 안 될까? 이것이 우리가 앞으로 할 일이다. 우리는 network가 원리대로라면 학습할 수 있는 latent semantic attribute 들을 얻게 될 것이다.

참고로 word embedding은 그 해석이 거의 불가능한 모델이다. 즉, 우리가 직접 정성껏 만든 vector들을 통해서 mathematician과 physicist가 둘 다 커피를 좋아하기 때문에 비슷하다고 해석할 수 있는 반면, neural network가 알아서 학습한 embedding을 봤을 때, 두 번째 차원에서 굉장히 큰 값을 둘 다 가지고 있음을 안다 해도 그 attribute가 어떤 의미를 지니는 지 알 길이 없는 것이다. 어떤 latent semantic dimension 상에서 비슷한 점이 있겠지만, 우리가 알 수 있는 것은 아닐 것이다.

정리하자면, **word embedding은 단어의 *의미*를 표현하는 것이고, 주어진 문제와 관련된 의미적 정보를 효율적으로 계산해서 이루어 진다.** 우리는 다른 것도 embed할 수 있다: 품사 정리, tree 쪼개기, 뭐든지! Feature embedding은 NLP에서 핵심적인 개념이다.

Word Embeddings in Pytorch
===========================================

예제와 숙제로 들어아기 전에, Pytorch에서, 그리고 일반적은 딥러닝 프로그래밍 상황에서 embedding을 사용하는 것에 대해 몇 가지 참고 사항을 잠깐 얘기하고자 한다. One-hot vector를 만들기 위해서 고유 index를 만들어줘야 하는 것처럼, embedding을 사용할 때도 단어 하나마다 고유한 index가 필요하다. 이 index가 lookup table의 키가 될 것이다. 즉, embedding은 $|V| \times D$ matrix에 저장이 될 것인데, 여기서 $D$는 embedding의 차원이고, index $i$에 해당하는 단어는 이 매트릭스의 $i$번째 행을 embedding으로 갈게 된다. 여기 제공되는 모든 코드에서 단어에서 index로 가는 mapping은 word_to_ix라는 Python dictionary다.

torch.nn.Embedding에서 제공하는 embedding은 두 가지 argument를 필요로 하는데, 단어장의 크기와 embedding의 차원 크기이다.

또한 lookup table에 대한 index로 사용하기 위해서는 반드시 torch.LongTensor를 사용해야 한다. Float 값은 index로 쓰일 수가 없다.

In [2]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [0]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings

In [0]:
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[-0.8923, -0.0583, -0.1955, -0.9656,  0.4224]],
       grad_fn=<EmbeddingBackward>)


In [0]:
for data in embeds.named_parameters():
    print(data)

('weight', Parameter containing:
tensor([[-0.8923, -0.0583, -0.1955, -0.9656,  0.4224],
        [ 0.2673, -0.4212, -0.5107, -1.5727, -0.1232]], requires_grad=True))


nn.Embedding 과 nn.Linear는 본질적으로 큰 차이가 없으나, Embedding은 입력을 index로 할 수 있다는 장점이 있다. 그리고 nn.Linear는 Embedding과 size가 transpose 되어 있다.

In [0]:
emb = nn.Embedding(3,5)
lin = nn.Linear(3,5, bias=False)

In [5]:
emb.weight

Parameter containing:
tensor([[ 0.2673, -0.4212, -0.5107, -1.5727, -0.1232],
        [ 3.5870, -1.8313,  1.5987, -1.2770,  0.3255],
        [-0.4791,  1.3790,  2.5286,  0.4107, -0.9880]], requires_grad=True)

In [6]:
lin.weight

Parameter containing:
tensor([[-0.3732,  0.3750,  0.3505],
        [ 0.5120, -0.3236, -0.0950],
        [-0.0112,  0.0843, -0.4382],
        [-0.4097,  0.3141, -0.1354],
        [ 0.2820,  0.0329,  0.1896]], requires_grad=True)

In [9]:
emb(torch.tensor([1], dtype=torch.long))

tensor([[ 3.5870, -1.8313,  1.5987, -1.2770,  0.3255]],
       grad_fn=<EmbeddingBackward>)

In [0]:
lin = nn.Linear(2,5,bias=False)

In [0]:
for data in lin.named_parameters():
    print(data)

('weight', Parameter containing:
tensor([[-0.4972, -0.5335],
        [ 0.0430, -0.1205],
        [ 0.4153, -0.4095],
        [-0.6286,  0.5146],
        [-0.1049,  0.3977]], requires_grad=True))


In [0]:
lin(torch.tensor([1.,0]))

tensor([-0.4972,  0.0430,  0.4153, -0.6286, -0.1049],
       grad_fn=<SqueezeBackward3>)

An Example: N-Gram Language Modeling
====================================

N-gram 모델에서 우리는 단어들의 나열 모음인 $w=(w_0,w_1,\ldots)$가 주어졌을 때, 과거 n개의 단어를 토대로 직후 등장할 단어의 확률을 예측하고자 한다.

\begin{align}P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )\end{align}

아래 예제에서 우리는 training 데이터를 바탕으로 loss 함수를 계산하고 backpropagation을 통해 parameter를 업데이트할 것이다.



In [0]:
CONTEXT_SIZE = 2 # 직전 두 단어를 토대로 예측하겠다.
EMBEDDING_DIM = 10
# 셰익스피어 소네트 2번을 훈련 데이터로 사용
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

# 직전 2단어와 현재 단어를 tuple로 묶자. 각 tuple은 다음과 같이 구성된다.
# ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# 예시
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [0]:
vocab = set(test_sentence) # 단어장 세팅
word_to_ix = {word: i for i, word in enumerate(vocab)} # 단어마다 index 매핑


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim) # 각 단어별 vector embedding
        self.linear1 = nn.Linear(context_size * embedding_dim, 128) # context vector의 embedding을 Linear mapping
        self.linear2 = nn.Linear(128, vocab_size) # 최종적으로는 각 단어의 예측 스코어 산출

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1)) # linear1의 input size에 매칭시키기 위해 형 변환
        out = F.relu(self.linear1(embeds)) 
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1) # 출력 스코어를 softmax를 통해 확률 분포로 변환
        return log_probs

In [0]:
losses = []
loss_function = nn.NLLLoss() # negative log loss 사용
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [0]:
for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[522.4741439819336, 520.1273083686829, 517.7977185249329, 515.4854674339294, 513.1894516944885, 510.90819478034973, 508.6412615776062, 506.38763093948364, 504.14531660079956, 501.91359782218933, 499.6916012763977, 497.4798996448517, 495.27676916122437, 493.0823287963867, 490.895583152771, 488.7174665927887, 486.5467276573181, 484.3849959373474, 482.22889399528503, 480.0796592235565]


Exercise: Computing Word Embeddings: Continuous Bag-of-Words
======================================

Continuous bag-of-words (CBOW) 모델은 NLP deep learning에서 자주 사용되는 방법으로, target 단어 앞 뒤로 몇 단어들의 문맥이 주어졌을 때에 알맞을 단어를 예측한다. CBOW는 순차적이지 않고 굳이 확률적일 필요가 없기 때문에 language modeling과 구별된다. CBOW를 통해 word embedding을 빠르게 학습하고, 더욱 복잡한 모델의 embedding을 위한 초기값으로 학습된 embedding을 사용하는 것이 전형적인 사용 예이다. 따라서 이 방법은 pretrained embeddings 라고 불리며, 1~2 퍼센트 정도의 성능 향상을 대부분의 경우 보장한다.

<img src='https://d2l.ai/_images/cbow.svg'>

CBOW 모델은 다음과 같다. Target 단어 $w_i$와 양 쪽 문맥 범위 $N$에 포함되는 $w_{i−1},\ldots,w_{i−N}$과 $w_{i+1},\ldots,w+{i+N}$이 주어졌을 때(이 문맥 단어들의 모음을 $C$라 하자), CBOW는 다음을 최소화하고자 한다.


\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}

여기서 $q_w$는 단어 $w$에 대한 embedding이다.

아래 Pytorch class를 채워서 CBOW 모델을 구성해보라. 팁을 주자면:

* 당신이 필요한 parameter가 무엇무엇인지 생각해보자.
* 각 연산이 필요로 하는 data의 모양이 무엇인지 확실하게 정리해라. 모양을 바꿔야 할 필요가 있다면 .view()를 사용하면 된다.




In [0]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()


vocab = set(raw_text) 
vocab_size = len(vocab) 
embedding_dim = 10

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [0]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size, bias=False)        

    def forward(self, inputs):
        embeds = self.embeddings(inputs).sum(0)
        out = self.linear(embeds).view(1,-1)
        return F.log_softmax(out, dim=1).view(1,-1)

여기서 embeds의 뒤에 sum(0)을 붙인 이유를 알아보자

In [0]:
emb = nn.Embedding(4,2)
emb.weight

Parameter containing:
tensor([[-0.3873, -0.9690],
        [ 0.0509,  0.7996],
        [ 1.4546,  0.7312],
        [-0.6275, -0.0908]], requires_grad=True)

In [0]:
emb(torch.tensor([1,2,3], dtype=torch.long)) # 각각 Embedding의 2,3,4번째 행 출력

tensor([[ 0.0509,  0.7996],
        [ 1.4546,  0.7312],
        [-0.6275, -0.0908]], grad_fn=<EmbeddingBackward>)

In [0]:
emb(torch.tensor([1,2,3], dtype=torch.long)).sum(0) # Embedding된 벡터의 component별 합산

tensor([0.8780, 1.4400], grad_fn=<SumBackward2>)

이제 모델을 생성하고 훈련시키자

In [0]:
model = CBOW(vocab_size, embedding_dim)

with torch.no_grad():
    print(model(torch.tensor([1,2],dtype=torch.long)))

tensor([[-4.0690, -4.1541, -4.6511, -4.3672, -4.1697, -3.4991, -3.6838, -4.7884,
         -5.4856, -5.9356, -3.3307, -4.3836, -3.1102, -3.1624, -4.7962, -3.6727,
         -4.5434, -5.1396, -2.4866, -4.9059, -2.9223, -4.0129, -3.1054, -5.1870,
         -3.4318, -4.7368, -4.8423, -5.0668, -4.9303, -4.2244, -4.6439, -5.1471,
         -3.0150, -4.8294, -5.3298, -4.9465, -4.5789, -4.0848, -4.4914, -4.1031,
         -4.4222, -4.1273, -2.9983, -3.8082, -5.0862, -3.6466, -5.0606, -2.8434,
         -3.0488]])


In [0]:
losses = []
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [0]:
# create your model and train.  here are some functions to help you make
# the data ready for use by your module

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

make_context_vector(data[0][0], word_to_ix) 

tensor([14, 40, 11, 47])

In [0]:
for epoch in range(10):
    total_loss = 0
    for context, target in data:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = make_context_vector(context, word_to_ix)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[242.1907639503479, 219.83423352241516, 200.63134288787842, 184.08665084838867, 169.77231788635254, 157.29002499580383, 146.2911548614502, 136.49880170822144, 127.7037365436554, 119.74704098701477]


PTB data set을 이용해서 훈련시켜보자

In [0]:
from pathlib import Path

DATA_PATH =  Path('Word2Vec') / 'Data'
File_name = 'ptb.train.txt'
print(DATA_PATH / File_name) # Path 설정 (운영체제 별로 / \ 혼용되기 때문)

Word2Vec\Data\ptb.train.txt


In [0]:
raw_text = open(DATA_PATH / File_name).read().replace('\n','<eos>').strip().split()

In [0]:
raw_text[:50]

['aer',
 'banknote',
 'berlitz',
 'calloway',
 'centrust',
 'cluett',
 'fromstein',
 'gitano',
 'guterman',
 'hydro-quebec',
 'ipo',
 'kia',
 'memotec',
 'mlx',
 'nahb',
 'punts',
 'rake',
 'regatta',
 'rubens',
 'sim',
 'snack-food',
 'ssangyong',
 'swapo',
 'wachter',
 '<eos>',
 'pierre',
 '<unk>',
 'N',
 'years',
 'old',
 'will',
 'join',
 'the',
 'board',
 'as',
 'a',
 'nonexecutive',
 'director',
 'nov.',
 'N',
 '<eos>',
 'mr.',
 '<unk>',
 'is',
 'chairman',
 'of',
 '<unk>',
 'n.v.',
 'the',
 'dutch']

In [0]:
CONTEXT_SIZE = 5
EMBEDDING_DIM = 100

# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)
embedding_dim = 10

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(5, len(raw_text) - 5):
    context = [raw_text[i - 5], raw_text[i - 4], raw_text[i - 3], raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2], raw_text[i + 3], raw_text[i + 4], raw_text[i + 5]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo'], 'cluett'), (['banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia'], 'fromstein'), (['berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec'], 'gitano'), (['calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx'], 'guterman'), (['centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'ipo', 'kia', 'memotec', 'mlx', 'nahb'], 'hydro-quebec')]


In [0]:
model = CBOW(vocab_size, embedding_dim)
losses = []
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [0]:
import time
start_tm = time.time()

for epoch in range(10):
    total_loss = 0
    for context, target in data:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!
tm = time.time() - start
print('Training time :', tm//3600,'H ', (tm%3600)//60, 'M ', (tm%3600)%60,'S ')

KeyboardInterrupt: 

In [0]:
print(losses)

[7572175.227815509, 7524815.120366931, 7516963.833378077]


In [0]:
class Skipgram(nn.Module):

    def __init__(self, vocab_size, embedding_dim):
        super(Skipgram, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size, bias=False)        

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        out = self.linear(embeds).view(1,-1)
        return F.log_softmax(out, dim=1)